# Семинар 09 - Ранжирование

In [1]:
import numpy as np
import pandas as pd

seed = 42
np.random.seed(seed)

D:\Anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<a id="Content"></a>
# Содержание
0. [Pointwise - TF-IDF](#0)
   - [Данные](#0.1)
   - [Неперсонализированная рекомендация](#0.2)
   - [Content-based рекомендация](#0.3)
1. [Pairwise - RankNet](#1)
2. [Listwise - ListNet](#2)
3. [Полезные ссылки](#3)

<a id="0"></a>
# 0. Pointwise - TF-IDF

<a id="0.1"></a>
## 0.1. Данные
[Исходники данных](https://www.kaggle.com/rounakbanik/the-movies-dataset) для работы на семинаре.

In [2]:
# загрузим данные о фильмах (набор документов)

metadata = pd.read_csv('data/movies/movies_metadata.csv', low_memory=False)
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
# загрузим данные о фильмах (оценки релевантности)
rating = pd.read_csv('data/movies/ratings_small.csv', low_memory=False)

rating.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


<a id="0.2"></a>
## 0.2. Неперсонализированная рекомендация

Сделаем свой аналог [IMDb rating](https://www.imdb.com/chart/top?ref_=nb_mv_3_chttp) через взвешенный рейтинг:
$$WeightedRating=(\frac{v}{v+m}⋅R)+(\frac{m}{v+m}⋅C)$$

где:
- v (votes) число оценок фильма;
- m (minimum) минимальное число оценок для попадания в топ;
- R (rating) средний рейтинг фильма;
- C (across) средний рейтинг по всем фильмам.

Эта формула позволяет сгладить рейтинги элементов с небольшим количеством оценок, приближая их к среднему рейтингу по всем элементам. Это помогает избежать ситуаций, когда элемент с небольшим количеством оценок, но высоким средним рейтингом, занимает слишком высокое место в рейтинге.

In [4]:
C = metadata['vote_average'].mean()
print(C)

m = metadata['vote_count'].quantile(0.90)
print(m)

q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
print(q_movies.shape)

5.618207215134185
160.0
(4555, 24)


**Задание 0:** Реализуйте взвешенную оценку. 

*Hint*: для того, чтобы применить функцию к столбцу с помощью метода .apply, реализуйте так, что входная переменная x является строкой с конкретным фильмом. 

In [5]:
def weighted_rating(x, m=m, C=C):

In [6]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [7]:
# Фильмы, основанные на баллах, рассчитанных выше
q_movies = q_movies.sort_values('score', ascending=False)

q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


Получилось достаточно близко к оригинальному топу.

<a id="0.3"></a>
## 0.3. Content-based рекомендация

Попробуем сделаем рекомендательную систему на основе описания фильмов.

In [8]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [9]:
first_n =  metadata.copy()[:30000]

### Найдем векторное представление описаний фильмов - TF-IDF

Рассмотрим частотное представление слов через TF-IDF.

TF-IDF (сокращение от term frequency — inverse document frequency) – это статистическая мера для оценки важности слова в документе, который является частью коллекции или корпуса.

Скоринг по TF-IDF растет пропорционально частоте появления слова в документе, но это компенсируется количеством документов, содержащих это слово.

Формула скоринга для слова X в документе Y:
![](img/td-idf-graphic.png)

TF (term frequency — частота слова) – отношение числа вхождений слова к общему числу слов документа.

![](img/tf.png)

IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции.

![](img/idf.png)

В итоге, вычислить TF-IDF для слова term можно так:

![](img/tf-idf.png)

Как это работает:

TF: Чем чаще слово встречается в документе, тем выше его TF. Например, если слово "кот" встречается 5 раз в документе, его TF будет выше, чем у слова "собака", которое встречается только 1 раз.

IDF: Чем реже слово встречается в коллекции документов, тем выше его IDF. Например, если слово "кот" встречается только в одном документе из 100, его IDF будет выше, чем у слова "и", которое встречается во всех документах.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
# ограничим размер словаря до 5000 элементов
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

In [12]:
first_n['overview'] = first_n['overview'].fillna('')

In [13]:
tfidf_matrix = tfidf.fit_transform(first_n['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(30000, 5000)

![](img/tf-idf_matrix.png)

In [14]:
tfidf.get_feature_names_out()[500:510]

array(['believe', 'believed', 'believes', 'believing', 'bell', 'belle',
       'belongs', 'beloved', 'ben', 'beneath'], dtype=object)

### Оценим схожесть полученных векторов

Схожесть будем измерять по косинусной метрике

In [20]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T).toarray()

In [21]:
cosine_sim.shape

(30000, 30000)

In [25]:
first_n['title']

0                                Toy Story
1                                  Jumanji
2                         Grumpier Old Men
3                        Waiting to Exhale
4              Father of the Bride Part II
                       ...                
29995                      Seventeen Again
29996                        Sweet Sixteen
29997    The Disappearance of Garcia Lorca
29998              The Dramatics: A Comedy
29999                         Up the Creek
Name: title, Length: 30000, dtype: object

In [26]:
indices = pd.Series(first_n.index, index=first_n['title']).drop_duplicates()

In [27]:
indices

title
Toy Story                                0
Jumanji                                  1
Grumpier Old Men                         2
Waiting to Exhale                        3
Father of the Bride Part II              4
                                     ...  
Seventeen Again                      29995
Sweet Sixteen                        29996
The Disappearance of Garcia Lorca    29997
The Dramatics: A Comedy              29998
Up the Creek                         29999
Length: 30000, dtype: int64

In [28]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Получить индекс фильма, соответствующий названию
    idx = indices[title]

    # Взять парные оценки сходства всех фильмов с этим фильмом
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Сортировать фильмы по сходству
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Взять оценки 10 самых похожих фильмов
    sim_scores = sim_scores[1:11]

    # Получить индексы фильма
    movie_indices = [i[0] for i in sim_scores]

    # Вернуть 10 самых похожих фильмов
    return metadata['title'].iloc[movie_indices]

In [29]:
get_recommendations('The Dark Knight Rises')

150                                         Batman Forever
1328                                        Batman Returns
12481                                      The Dark Knight
21194    Batman Unmasked: The Psychology of the Dark Kn...
20232              Batman: The Dark Knight Returns, Part 2
15511                           Batman: Under the Red Hood
11753                                            Slow Burn
27521                                       Rage of Angels
6042                                                 Q & A
4363                                          Criminal Law
Name: title, dtype: object

### Сделеаем рекомендацию для пользователя

In [30]:
rating = rating.drop(['timestamp'], axis=1)
rating

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [31]:
# возьмем одного пользователя
user_id = 1

user_rating = rating[rating['userId'] == user_id].drop(['userId'], axis=1).sort_values(by=['rating'], ascending=False)
user_rating

,movieId,rating
4,1172,4.0
13,2105,4.0
12,1953,4.0
8,1339,3.5
19,3671,3.0
1,1029,3.0
2,1061,3.0
14,2150,3.0
17,2455,2.5
0,31,2.5


In [32]:
user_rating["movieId"].values[0]

1172

In [34]:
first_n["title"].values[1172]

'Army of Darkness'

In [35]:
indices["Army of Darkness"]

1172

In [36]:
get_recommendations("Army of Darkness")

17978                                     Return of Django
22714                                      Berkeley Square
8618                                           Point Blank
3319     Teenage Mutant Ninja Turtles II: The Secret of...
16736                                     Run of the Arrow
7215                                       Never Die Alone
24326                                     No Time for Nuts
20971                                                 Epic
10716                                           Hallelujah
13373                                      Point of Order!
Name: title, dtype: object

In [38]:
first_n["title"].values[user_rating["movieId"].values]

array(['Army of Darkness', 'Number Seventeen', 'The Governess', 'Jaws 2',
       'X-Men', 'Romeo + Juliet', 'The Doors', 'About Last Night...',
       'The Mod Squad', 'Twelve Monkeys', 'Inside', 'Citizen Ruth',
       'Burnt Offerings', 'The Blob', 'Parallel Sons', 'Holy Man',
       'Cocoon', 'American Dream',
       'Unforgotten: Twenty-Five Years After Willowbrook',
       'The Living Dead Girl'], dtype=object)

<a id="1"></a>
# 1. Pairwise - RankNet

Функция ошибки по паре объектов (в пару к запросу).

$\displaystyle \sum_q \sum_{i, j:\ r^q_i \gt r^q_j} l(f({x}^q_i) - f({x}^q_j)) \to \min$

В качестве функции для оптимизации мы берём классическую функцию потерь: кросс-энтропию $C$:
$C_{ij}=C(o_{ij})=-\bar{P_{ij}}log(P_{ij})-(1-\bar{P_{ij}})log(1-P_{ij})$

$o_i$ — предсказание нашего алгоритма для одного объекта (*логит* или *скор*):

$o_i \equiv f(x_i)$,

$o_{ij}=f(x_i)-f(x_j)$

Для превращения этого в вероятность, т.е. нормирования в интервал $[0, 1]$, мы можем воспользоваться обычной логистической функцией. Разность логитов будем использовать как степень для числа $e$:

$\displaystyle P_{ij} \equiv \frac {e^{o_{ij}}} {1 + e^{o_{ij}}}$ — функция отображения предсказания (логита) в вероятность.


Тогда функцию потерь, или функцию стоимости (cost function) можно переписать следующим образом:

$C_{ij} = -\overline P_{ij} o_{ij} + \log(1 + e^{o_{ij}})$

In [15]:
import torch

In [16]:
class RankNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )
        
        self.out_activation = torch.nn.Sigmoid()

    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)

        return out
    
    def predict(self, inp):
        logits = self.model(inp)
        return logits

In [17]:
ranknet_model = RankNet(num_input_features=10)

In [19]:
inp_1, inp_2 = torch.rand(4, 10), torch.rand(4, 10)
# batch_size x input_dim
inp_2, inp_1

(tensor([[0.3544, 0.7896, 0.7779, 0.5681, 0.9910, 0.0394, 0.6498, 0.9910, 0.8920,
          0.1686],
         [0.7035, 0.8068, 0.2411, 0.0703, 0.6117, 0.0247, 0.9841, 0.8135, 0.1979,
          0.5919],
         [0.4757, 0.2316, 0.8592, 0.4112, 0.3345, 0.1644, 0.2501, 0.9897, 0.0430,
          0.5650],
         [0.5516, 0.3083, 0.7735, 0.5606, 0.2462, 0.5510, 0.6025, 0.6693, 0.6546,
          0.4504]]),
 tensor([[0.6360, 0.1272, 0.3292, 0.5305, 0.3082, 0.7333, 0.5952, 0.7116, 0.8484,
          0.9124],
         [0.2592, 0.9663, 0.1726, 0.6058, 0.1518, 0.9204, 0.8926, 0.6894, 0.0559,
          0.4377],
         [0.0274, 0.4307, 0.2349, 0.5437, 0.9654, 0.7846, 0.2720, 0.6678, 0.7380,
          0.6404],
         [0.5313, 0.9752, 0.5979, 0.6314, 0.5449, 0.5772, 0.7725, 0.0698, 0.5350,
          0.4211]]))

In [23]:
preds = ranknet_model(inp_1, inp_2)
preds

tensor([[0.5424],
        [0.5143],
        [0.4981],
        [0.5075]], grad_fn=<SigmoidBackward0>)

In [24]:
first_linear_layer = ranknet_model.model[0]

In [25]:
first_linear_layer.weight.grad

In [26]:
criterion = torch.nn.BCELoss()
loss = criterion(preds, torch.ones_like(preds))
loss.backward()

In [27]:
first_linear_layer.weight.grad

tensor([[ 0.0051, -0.0056, -0.0112, -0.0175,  0.0018, -0.0173,  0.0062,  0.0229,
         -0.0106,  0.0051],
        [-0.0016, -0.0005, -0.0017, -0.0005, -0.0008, -0.0002, -0.0014, -0.0031,
         -0.0005, -0.0016],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0085, -0.0053,  0.0101, -0.0016,  0.0045, -0.0067,  0.0016,  0.0156,
          0.0027,  0.0068],
        [-0.0074,  0.0047, -0.0147,  0.0079, -0.0017,  0.0247,  0.0006, -0.0148,
          0.0046,  0.0066],
        [-0.0196,  0.0123, -0.0388,  0.0209, -0.0044,  0.0655,  0.0015, -0.0392,
          0.0122,  0.0176],
        [-0.0250,  0.0157, -0.0496,  0.0267, -0.0056,  0.0836,  0.0020, -0.0501,
          0.0156,  0.0224],
        [-0.0104,  0.0091, -0.0125,  0.0115, -0.0028,  0.0310,  0.0015, -0.0196,
          0.0041,  0.0028],
        [-0.0028, -0.0201, -0.0013, -0.0233, -0.0028, -0.0340, -0.0135,  0.0013,
         -0.0113, -0.0142],
        [ 0.0254, -

In [28]:
ranknet_model.zero_grad()

<a id="2"></a>
# 2. Listwise - ListNet

В listwise, как следует из названия, мы должны использовать функцию потерь, которая рассчитывается на всём множестве релевантных запросу документов.

Можем говорить, что представлено полное множество перестановок $\Omega_n$, указывая размер множества объектов, на которых рассчитываются перестановки $n$.

Каждая перестановка $\pi$  характеризуется полным указанием, какой объект стоит на первой, на второй и так далее до позиции $n$.

$\pi = \langle \pi(1), \pi(2), ..., \pi(n) \rangle$

Каждое $\pi_i$ указывает на конкретный объект в перестановке.

$\displaystyle P_s (\pi) = \prod^n_{j = 1} \frac {\phi(s_{\pi(j)})} {\sum^n_{k = j} \phi(s_{\pi(k)})}$ — вероятность возникновения такой перестановки

И в числителе, и в знаменателе к скору, или к логиту, $j$-го объекта конкретной перестановки $\pi_i$ применяется функция преобразования скоров.

К этой функции указываются следующие требования:

- Возрастающая;
- Строго положительная.

То есть, чем больше логит, тем выше значение этой функции, при этом ни при каких обстоятельствах она не может стать отрицательной (*иначе бы мы могли получать отрицательные вероятности, чего быть не может*).

Под эти требования подходит много функций, но самая распространенная — экспонента, то есть возведение e в степень логита с индексом $\pi_j$.

Рассмотрим знаменатель: здесь сумма от $j$-го до $n$-го (последнего) объекта, суммируем мы в точности те же значения, что и в числителе — некоего рода нормализация.

Смотрим, какую долю от суммы всех скоров составляет наш текущий $j$-й объект.

![](img/softmax.png)

Выводы для метода:

- Наибольшая вероятность у перестановки, в которой объекты отсортированы в порядке убывания.
- Наименьшая вероятность у перестановки, в которой объекты отсортированы в порядке возрастания.
- Количество перестановок равно $n!$ (много).

Благодаря SoftMax не нужно считать все перестановки — можно получить скоры и преобразовать их в TopOneProbability.

Для обучения нашего алгоритма, для расчёта градиентов и градиентного спуска по весам, можно использовать любую функцию потерь, которая оптимизирует расстояние между двумя распределениями вероятностей.

Это может быть классическая кросс-энтропия:

$\displaystyle L(y^{(i)}, z^{(i)}) = -\sum^n_{j = 1} P_{y^{(i)}}(j) \log(P_{z^{(i)}}(j))$,

In [29]:
from itertools import combinations

from utils import ndcg, num_swapped_pairs

In [30]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )


    def forward(self, input_1):
        logits = self.model(input_1)
        return logits

In [31]:
def listnet_ce_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """

    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i))

def listnet_kl_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))


def make_dataset(N_train, N_valid, vector_dim):
    fake_weights = torch.randn(vector_dim, 1)

    X_train = torch.randn(N_train, vector_dim)
    X_valid = torch.randn(N_valid, vector_dim)

    ys_train_score = torch.mm(X_train, fake_weights)
    ys_train_score += torch.randn_like(ys_train_score)

    ys_valid_score = torch.mm(X_valid, fake_weights)
    ys_valid_score += torch.randn_like(ys_valid_score)

#     bins = [-1, 1]  # 3 relevances
    bins = [-1, 0, 1, 2]  # 5 relevances
    ys_train_rel = torch.Tensor(
        np.digitize(ys_train_score.clone().detach().numpy(), bins=bins)
    )
    ys_valid_rel = torch.Tensor(
        np.digitize(ys_valid_score.clone().detach().numpy(), bins=bins)
    )

    return X_train, X_valid, ys_train_rel, ys_valid_rel

In [35]:
N_train = 1000
N_valid = 500

vector_dim = 100
epochs = 2

batch_size = 16

X_train, X_valid, ys_train, ys_valid = make_dataset(N_train, N_valid, vector_dim)

net = ListNet(num_input_features=vector_dim)
opt = torch.optim.Adam(net.parameters())

In [36]:
torch.unique(ys_train)

tensor([0., 1., 2., 3., 4.])

In [37]:
for epoch in range(epochs):
    idx = torch.randperm(N_train)

    X_train = X_train[idx]
    ys_train = ys_train[idx]

    cur_batch = 0
    for it in range(N_train // batch_size):
        batch_X = X_train[cur_batch: cur_batch + batch_size]
        batch_ys = ys_train[cur_batch: cur_batch + batch_size]
        cur_batch += batch_size

        opt.zero_grad()
        if len(batch_X) > 0:
            batch_pred = net(batch_X)
            batch_loss = listnet_kl_loss(batch_ys, batch_pred)
            # batch_loss = listnet_ce_loss(batch_ys, batch_pred)
            batch_loss.backward(retain_graph=True)
            opt.step()

        if it % 10 == 0:
            with torch.no_grad():
                valid_pred = net(X_valid)
                valid_swapped_pairs = num_swapped_pairs(ys_valid, valid_pred)
                ndcg_score = ndcg(ys_valid, valid_pred)
            print(f"epoch: {epoch + 1}.\tNumber of swapped pairs: " 
                  f"{valid_swapped_pairs}/{N_valid * (N_valid - 1) // 2}\t"
                  f"nDCG: {ndcg_score:.4f}")

epoch: 1.	Number of swapped pairs: 43485/124750	nDCG: 0.8082
epoch: 1.	Number of swapped pairs: 41090/124750	nDCG: 0.8386
epoch: 1.	Number of swapped pairs: 38912/124750	nDCG: 0.8495
epoch: 1.	Number of swapped pairs: 36422/124750	nDCG: 0.8656
epoch: 1.	Number of swapped pairs: 33965/124750	nDCG: 0.8784
epoch: 1.	Number of swapped pairs: 31494/124750	nDCG: 0.8940
epoch: 1.	Number of swapped pairs: 29122/124750	nDCG: 0.9076
epoch: 2.	Number of swapped pairs: 28723/124750	nDCG: 0.9100
epoch: 2.	Number of swapped pairs: 26520/124750	nDCG: 0.9209
epoch: 2.	Number of swapped pairs: 24144/124750	nDCG: 0.9314
epoch: 2.	Number of swapped pairs: 22032/124750	nDCG: 0.9395
epoch: 2.	Number of swapped pairs: 20209/124750	nDCG: 0.9469
epoch: 2.	Number of swapped pairs: 18535/124750	nDCG: 0.9543
epoch: 2.	Number of swapped pairs: 17184/124750	nDCG: 0.9596


<a id="3"></a>
# 3. Полезные ссылки
Что почитать:
1. [От сопоставления слов к TF-IDF и до BM-25](https://habr.com/ru/articles/823568/)

Что посмотреть:
1. [Лекции](https://disk.yandex.ru/d/VTH7icXZRpXgAA)